# Image Scraper

As noted before, this code is no longer functional for the website it was intended. However, please look at the work that was put into this portion of my project. As written about in my paper, this code did work, and I was able to download a small sample of images before the change was made. 

In [181]:
import pandas as pd
import numpy as np

import string

import re

import os
## import the urllib stuff for scraping

import urllib
from bs4 import BeautifulSoup

In [101]:
#Artchive - cant use, too variable
artchive_page = "http://www.artchive.com/artchive/artists.html"
artchive_lis = "http://www.artchive.com/artchive/"

#WikiArt
wikiart_page = "https://www.wikiart.org/en/alphabet" #each artist is listed in the alphabet
wikiart = "https://www.wikiart.org"
wikiart_genre = "https://www.wikiart.org/en/artists-by-genre"
wikiart_rel = "/religious-painting"


imgPath = "Wikiart"
# aPage = urllib.request.urlopen(artchive_page)

# soup = BeautifulSoup(aPage, 'html5lib')

In [13]:
wApage = urllib.request.urlopen(wikiart_page)

soup = BeautifulSoup(wApage, 'html5lib')

In [15]:
lis = soup.find_all('ul', attrs={'class': 'alphabet-list'})

## Scraping for Artist pages

These functions are used to get the pages for each artist, either all of them on the site, or all for a genre. 

#### Please note that all the functions are depricated. Wikiart.org changed their site, and none of this works. 

In [231]:
#get all artists and their page links
def getAllArtists():
    al = {}
    for c in string.ascii_lowercase:
        page = urllib.request.urlopen(wikiart_page + '/' + c)
        soup = BeautifulSoup(page, 'html5lib')

        ul = soup.find_all('ul', attrs={'class': 'wiki-artistgallery-container ng-isolate-scope'})
        print(c, len(ul))

        for link in ul[0].find_all('a', attrs={'href': True, 'title': True, 'style': True}):
            #name = link['title']
            name = link['href'][4:].replace('-', ' ')
            al[name] = link['href']
    return al

def getGenre(genre):
    al = {}
    page = urllib.request.urlopen(wikiart_genre + genre)
    soup = BeautifulSoup(page, 'html5lib')

    div = soup.find_all('div', attrs={'class': 'artist-gallery-content'})

    print(len(div))
    for link in div[0].find_all('a', attrs={'href': True, 'class': True}):
        print(link)
        #name = link['title']
        name = link['href'][4:].replace('-', ' ')
        al[name] = link['href']
    
    return al

religiousArtists = getGenre(wikiart_rel)

1
<a class="image-wrapper" href="/en/alberto-sotio">
                                    <img src="https://uploads.wikiart.org/Content/images/ARTIST-480x600.jpg" title="Alberto Sotio"/>
                                </a>
<a class="image-wrapper" href="/en/duccio">
                                    <img src="https://uploads.wikiart.org/Content/images/ARTIST-480x600.jpg" title="Duccio"/>
                                </a>
<a class="image-wrapper" href="/en/pietro-lorenzetti">
                                    <img src="https://uploads.wikiart.org/Content/images/ARTIST-480x600.jpg" title="Pietro Lorenzetti"/>
                                </a>
<a class="image-wrapper" href="/en/simone-martini">
                                    <img src="https://uploads3.wikiart.org/images/simone-martini.jpg!PinterestLarge.jpg" title="Simone Martini"/>
                                </a>
<a class="image-wrapper" href="/en/paolo-veneziano">
                                    <img src="https:/

In [232]:
len(religiousArtists)

42

In [73]:
artist_links = getAllArtists()

a 1
b 1
c 1
d 1
e 1
f 1
g 1
h 1
i 1
j 1
k 1
l 1
m 1
n 1
o 1
p 1
q 1
r 1
s 1
t 1
u 1
v 1
w 1
x 1
y 1
z 1


In [75]:
len(artist_links)

2814

## Scraping for Artwork



In [283]:
#artist name '/all-works' gets you all their artworks
artist_painting_links = {}
def getPaintingLinks(artistLinks):
    apl = {}
    for n in list(artistLinks.keys())[:5]:
        paintings = {}
        artistPage = artistLinks[n]

        page = urllib.request.urlopen(wikiart + artistPage + "/all-works")
        soup = BeautifulSoup(page, 'html5lib')
        
        art_links = soup.find_all('div', attrs={'class': 'title-block'})
        print(art_links)
#         for ul in art_links:
#             art = ul.find('a', attrs={'class': True})
#             if art['href'] != artistPage:
#                 name = art.getText()
#                 paintings[name] = art['href']
#         apl[n] = paintings
            
    return apl       

In [282]:
rel_paintLinks = getPaintingLinks(religiousArtists)

[]
[]
[]
[]
[]


In [249]:
rel_paintLinks

{'alberto sotio': {},
 'duccio': {},
 'paolo veneziano': {},
 'pietro lorenzetti': {},
 'simone martini': {}}

In [82]:
page = urllib.request.urlopen(wikiart + rel_paintLinks['alberto sotio']['Madonna Di Ambro'])
soup = BeautifulSoup(page, 'html5lib')

In [87]:
soup.find_all('a', attrs={'id':'image-link'})[0]['href']

'https://uploads6.wikiart.org/00114/images/alberto-sotio/madonna-di-ambro.jpg'

In [208]:
def downloadImgs(artistPaint, numPaintings=5):
    paintings = {}
    count = 0   
    for n in list(artistPaint.keys())[:5]:
        aName = len(n)
        paintings_num = 1
        if len(artistPaint[n]) >= numPaintings:
            paintings_num = numPaintings
        else:
            paintings_num = len(artistPaint[n])
    
        for p in list(artistPaint[n].keys())[:paintings_num]:
            count += 1
            page = urllib.request.urlopen(wikiart + artistPaint[n][p])
            soup = BeautifulSoup(page, 'html5lib')
            #print(artistPaint[n][p])

            link = soup.find('img', attrs={'itemprop': 'image'})['src']

            link = urllib.parse.quote(link.encode('utf8'), ':/')
            # need to get the image downloaded, get the year (if it exisits), get genre
            dirpath = imgPath + '/' + n.replace(' ', '-')
            try: 
                os.mkdir(dirpath)
            except FileExistsError:
                pass
                #print('Directory {} already exists'.format(dirpath))

            path = urllib.request.urlretrieve(link, (dirpath + '/' + artistPaint[n][p][5 + aName:] + '.jpg'))
            year = getVal(soup, 'dateCreated')
            genre = getVal(soup, 'genre')
            print(p)
            paintings[p + ' ' + n] = [n, artistPaint[n][p], path[0], year, genre]
    print(len(paintings), count)
    
    return paintings
    
    
#Fix this function, for the proper way to get it out
# not all the paintings have this same layout for 
# genre, and if I wan to add style. 
def getVal(s, t):
    r = s.find('span', attrs={'itemprop': t})
    if r:
        return r.getText()
    return None

In [209]:
rel_paintD = downloadImgs(rel_paintLinks, 5)

Madonna in Maesta
The Crucifixion with the Virgin Mary and St John
The Mourning Virgin Mary 1180 90
Martyrdom of St. John and St. Paul
Madonna Di Ambro
Crevole Madonna
Madonna and Child (no. 593)
Maestà
Madonna and Child
Madonna of Crevole
Polyptych
Crucifix
Shaped Cross
Predella Panel. St Albert Presents the Rule to the Carmelites
Predella Panel. Hermits at the Fountain of Elijah
Saint Catherine of Alexandria Polyptych
Maestà
Virgin and Child with Saints (Boston Polyptych)
Guidoriccio da Fogliano
The Miracle of the Child Attacked and Rescued by Augustine Novello
Coronation of the Virgin
Madonna and Child with Two Votaries
Madonna of the Poppy
Doge Francesco Dandolo and His Wife Presented to the Madonna
Monument to the Doge Francesco Dandolo
25 25


In [210]:
rel_paintDF = pd.DataFrame.from_dict(rel_paintD).T.rename(columns={0:"Artist", 1:"Link Ext", 2:"Path", 3:"Year", 4:"Genre"})

In [211]:
rel_paintDF

,Artist,Link Ext,Path,Year,Genre
Coronation of the Virgin paolo veneziano,paolo veneziano,/en/paolo-veneziano/coronation-of-the-virgin-1324,Wikiart/paolo-veneziano/coronation-of-the-virg...,1324,religious painting
Crevole Madonna duccio,duccio,/en/duccio/crevole-madonna,Wikiart/duccio/crevole-madonna.jpg,1280,religious painting
Crucifix pietro lorenzetti,pietro lorenzetti,/en/pietro-lorenzetti/crucifix,Wikiart/pietro-lorenzetti/crucifix.jpg,1320,religious painting
Doge Francesco Dandolo and His Wife Presented to the Madonna paolo veneziano,paolo veneziano,/en/paolo-veneziano/doge-francesco-dandolo-and...,Wikiart/paolo-veneziano/doge-francesco-dandolo...,1339,religious painting
Guidoriccio da Fogliano simone martini,simone martini,/en/simone-martini/guidoriccio-da-fogliano-1328,Wikiart/simone-martini/guidoriccio-da-fogliano...,1328,None
Madonna Di Ambro alberto sotio,alberto sotio,/en/alberto-sotio/madonna-di-ambro,Wikiart/alberto-sotio/madonna-di-ambro.jpg,None,religious painting
Madonna and Child (no. 593) duccio,duccio,/en/duccio/madonna-and-child-no-593,Wikiart/duccio/madonna-and-child-no-593.jpg,1280,religious painting
Madonna and Child duccio,duccio,/en/duccio/madonna-and-child-1305,Wikiart/duccio/madonna-and-child-1305.jpg,1305,religious painting
Madonna and Child with Two Votaries paolo veneziano,paolo veneziano,/en/paolo-veneziano/madonna-and-child-with-two...,Wikiart/paolo-veneziano/madonna-and-child-with...,1325,religious painting
Madonna in Maesta alberto sotio,alberto sotio,/en/alberto-sotio/madonna-in-maesta,Wikiart/alberto-sotio/madonna-in-maesta.jpg,None,religious painting


In [212]:
dRel_paintDF = rel_paintDF.dropna()

In [213]:
dRel_paintDF = dRel_paintDF.sort_values(by='Year', ascending=True)

In [214]:
dRel_paintDF

,Artist,Link Ext,Path,Year,Genre
Maestà duccio,duccio,/en/duccio/maesta-1280,Wikiart/duccio/maesta-1280.jpg,1280,religious painting
Crevole Madonna duccio,duccio,/en/duccio/crevole-madonna,Wikiart/duccio/crevole-madonna.jpg,1280,religious painting
Madonna and Child (no. 593) duccio,duccio,/en/duccio/madonna-and-child-no-593,Wikiart/duccio/madonna-and-child-no-593.jpg,1280,religious painting
Madonna of Crevole duccio,duccio,/en/duccio/madonna-of-crevole-1284,Wikiart/duccio/madonna-of-crevole-1284.jpg,1284,religious painting
Madonna and Child duccio,duccio,/en/duccio/madonna-and-child-1305,Wikiart/duccio/madonna-and-child-1305.jpg,1305,religious painting
Saint Catherine of Alexandria Polyptych simone martini,simone martini,/en/simone-martini/saint-catherine-of-alexandr...,Wikiart/simone-martini/saint-catherine-of-alex...,1319,religious painting
Crucifix pietro lorenzetti,pietro lorenzetti,/en/pietro-lorenzetti/crucifix,Wikiart/pietro-lorenzetti/crucifix.jpg,1320,religious painting
Polyptych pietro lorenzetti,pietro lorenzetti,/en/pietro-lorenzetti/polyptych-1320,Wikiart/pietro-lorenzetti/polyptych-1320.jpg,1320,religious painting
Maestà simone martini,simone martini,/en/simone-martini/maest-1321,Wikiart/simone-martini/maest-1321.jpg,1321,religious painting
Coronation of the Virgin paolo veneziano,paolo veneziano,/en/paolo-veneziano/coronation-of-the-virgin-1324,Wikiart/paolo-veneziano/coronation-of-the-virg...,1324,religious painting


In [215]:
dRel_paintDF.to_csv('rel_paint.csv', encoding='utf-8')

In [217]:
for index, row in dRel_paintDF.iterrows():
    print(row['Path'])

Wikiart/duccio/maesta-1280.jpg
Wikiart/duccio/crevole-madonna.jpg
Wikiart/duccio/madonna-and-child-no-593.jpg
Wikiart/duccio/madonna-of-crevole-1284.jpg
Wikiart/duccio/madonna-and-child-1305.jpg
Wikiart/simone-martini/saint-catherine-of-alexandria-polyptych-1319.jpg
Wikiart/pietro-lorenzetti/crucifix.jpg
Wikiart/pietro-lorenzetti/polyptych-1320.jpg
Wikiart/simone-martini/maest-1321.jpg
Wikiart/paolo-veneziano/coronation-of-the-virgin-1324.jpg
Wikiart/paolo-veneziano/madonna-of-the-poppy-1325.jpg
Wikiart/paolo-veneziano/madonna-and-child-with-two-votaries-1325.jpg
Wikiart/pietro-lorenzetti/shaped-cross-1325.jpg
Wikiart/simone-martini/virgin-and-child-with-saints-boston-polyptych-1325.jpg
Wikiart/simone-martini/the-miracle-of-the-child-attacked-and-rescued-by-augustine-novello-1328.jpg
Wikiart/pietro-lorenzetti/predella-panel-hermits-at-the-fountain-of-elijah-1329.jpg
Wikiart/pietro-lorenzetti/predella-panel-st-albert-presents-the-rule-to-the-carmelites-1329.jpg
Wikiart/paolo-veneziano/m

In [287]:
import json
l = "https://www.wikiart.org/en/alberto-sotio/all-works"


page = urllib.request.urlopen(l).read()
soup = BeautifulSoup(page, 'html5lib')


74359

In [292]:
x = soup.get_text().find("title-block")

In [294]:
soup.get_text()[x-100:x+50]

'ay{}.wiki-masonry-container>li .supersized-hover-container{display:none}.wiki-masonry-container>li .title-block .like-overlay{display:none}@media(min-'

In [304]:
soup.find_all('ul', attrs ={'class': 'wiki-masonry-container'})

[<ul class="wiki-masonry-container" masonry-items="paintingsToDisplay" masonry-options="options" masonry-view="">
             </ul>]

In [303]:
soup.select('.title-block')

[]

In [309]:
np.array(2)

array(2)